结合爬虫和多agent原理，我们可以定制一个可以根据用户的问题进行多角度股票分析的智能体。

当我们遇见一个股票类问题，比如：立讯精密的财务报表数据和舆情怎么样？
我们的StockAgent工作流程可以是如下：
1. 分析这个问题与哪只股票相关；
2. 提取这只股票对应的代码；
3. 分析这个问题需要用到哪些数据，然后逐个提取并汇总；
4. 紧接着喂给LLM进行一个最终的分析。

在这里我们需要为Agent提供的主要工具就是爬虫，集成到get_stock_info_llm_needed这个工具函数中。

最终我们实现的效果就是，我们的AI可以根据用户的问题，自由获取股票分析所需要的数据，然后据此回答问题。

可以优化的地方：
- 增加更多数据和Agent自由度，让Agent能自主选择工具回答更多类型的问题
- 增加code interpreter工具，可以让Agent根据已有数据构建财务估值模型计算公司合理估值
...

欢迎关注，一起成为AI先行者~
agent-101项目updated~



In [1]:
from tools.llm_api import *
from tools.json_tool import *
from free_agent.stock_agent.stock_agent_tools.get_comment_list import get_comment_list
from free_agent.stock_agent.stock_agent_tools.get_recent_financial_statments import get_recent_financial_statments
from selenium.webdriver.chrome.options import Options

"""
问题 
↓(cmd_judge_prompt)
判断需求（提取股票代码 + 问题分类）
↓
→ 财务分析 → 提取表格重要指标 
→ 舆情分析
↓

"""

class StockAgent:
    def __init__(self, agent_name, model="glm-4", temperature=0.8):
        self.agent_name = agent_name
        self.model = model
        self.temperature = temperature
        # prompt: 提取股票名称
        self.get_stock_name_prompt = self.get_prompt("free_agent/stock_agent/stock_agent_prompts/get_stock_name_prompt.txt")
        # prompt: 判断需求
        self.cmd_judge_prompt = self.get_prompt("free_agent/stock_agent/stock_agent_prompts/cmd_judge_prompt.txt")
        self.system_prompt = self.get_prompt("free_agent/stock_agent/stock_agent_prompts/system_prompt.txt")
        self.total_conversation =[{"role": "system", "content": self.system_prompt}]
        
    def get_driver(self):
        options = Options()  
        options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"')
        options.add_argument("--headless")  
        driver = webdriver.Chrome(options=options)  
        return driver
    
    def get_prompt(self, path):
        with open(path, 'r', encoding="utf-8") as file:
            prompt = file.read()
        return prompt

    def get_stock_index_by_stockname(self, stock_name):
        stock_indexes = pd.read_excel("free_agent/stock_agent/stock_indexes.xlsx", index_col=False)
        stock_indexes["证券名称"] = stock_indexes["证券名称"].str.replace(" ", "")
        stock_indexes["indexes"] = stock_indexes["证券代码"].str[-2:] + stock_indexes["证券代码"].str[:6] 
        stock_index = stock_indexes.loc[stock_indexes["证券名称"].str.contains(stock_name), "indexes"].values[0]
        return stock_index

    # 提取股票代码
    def get_stock_index(self, question):
        get_stock_name_prompt = self.get_stock_name_prompt.format(question=question)
        ans = ""
        for char in get_llm_answer(question=get_stock_name_prompt, model="glm-4"):
            ans += char
            print(char, end="", flush=True)
        stock_name = get_json(ans)["stock_name"]
        stock_index = self.get_stock_index_by_stockname(stock_name)
        return stock_index

    # 根据问题分类提取数据
    def get_stock_info_by_qtype(self, driver, question_type, stock_index):
        if question_type == "财务分析":
            data = get_recent_financial_statments(driver, stock_index)
            return data
        elif question_type == "股吧舆情":
            data = get_comment_list(driver, stock_index, page=1)
            return data
        else:
            return ""

    def judge_question_type(self, question):
        print("\n\n")
        cmd_judge_prompt = self.cmd_judge_prompt.format(question=question)
        ans = ""
        for char in get_llm_answer(question=cmd_judge_prompt, model="glm-4"):
            ans += char
            print(char, end="", flush=True)
        question_type = get_json(ans)["question_type"]
        print("\n\n" + str(question_type))
        return question_type

    def get_stock_info_llm_needed(self, question):
        stock_index = self.get_stock_index(question)
        question_type = self.judge_question_type(question)
        driver = self.get_driver()
        total_data = ""
        for q_type in question_type:
            data = self.get_stock_info_by_qtype(driver, q_type, stock_index)
            data = f"{q_type}相关信息：\n\n" + str(data) + "\n" + "-"*50 + "\n"
            total_data += data
        driver.quit()
        return total_data

    def agent_work_flow(self, question):
        stock_info = self.get_stock_info_llm_needed(question)
        self.total_conversation.append({"role": "user", "content": question})
        self.total_conversation.append({"role": "assistant", "content": f"根据你的问题，我找到了以下相关信息：{stock_info}"})
        self.total_conversation.append({"role": "user", "content": f"请尽可能详细分析上述信息回答我的问题"})
        ans = ""
        for char in get_llm_answer_converse(self.total_conversation, self.model, self.temperature):
            print(char, end="", flush=True)
        return ans
        

In [17]:
question = "立讯精密的财务报表数据怎么样"
stock_agent = StockAgent("stock_agent")
stock_agent.agent_work_flow(question)

```json
{
    "stock_name": "立讯精密"
}
```


```json
{
    "question_type": ["财务分析", "公司分析"]
}
```

['财务分析', '公司分析']
sz002475的最近财务报表数据...
正在访问链接...
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=sz002475&color=b#/cwfx
数据长度:44
根据提供的数据，我将分析立讯精密的财务状况。为了提供一个全面的财务分析，我会分几个关键部分来评估公司的财务状况。

### 1. 收益和盈利能力

#### 关键指标
- 营业总收入：24Q1为524.1亿，同比增长4.93%
- 毛利润：24Q1为56.31亿，毛利率10.74%
- 归属净利润：24Q1为24.71亿，同比增长22.45%
- 扣非净利润：24Q1为21.83亿，同比增长23.23%

#### 分析
立讯精密的营业总收入和净利润在最近几个季度呈现增长趋势，显示出公司的整体盈利能力在增强。毛利率相对稳定，保持在10%以上，表明公司产品有一定的利润空间。

### 2. 财务健康状况

#### 关键指标
- 流动比率：24Q1为1.388
- 速动比率：24Q1为1.010
- 资产负债率：24Q1为57.12%
- 权益系数：24Q1为2.332

#### 分析
公司的流动比率和速动比率均大于1，表明短期偿债能力良好。资产负债率虽然较高，但仍在可控范围内，显示出公司有一定的财务杠杆。

### 3. 成长能力

#### 关键指标
- 营业总收入同比增长：24Q1为4.93%
- 归属净利润同比增长：24Q1为22.45%
- 扣非净利润同比增长：24Q1为23.23%

#### 分析
立讯精密的营收和净利润同比增长率均保持在较高水平，特别是净利润的增长，显示出公司具有较强的成长能力。

### 4. 营运能力

#### 关键指标
- 总资产周转率：24Q1为0.315
- 存货周转率：24Q1为1.625
- 应收账款周转率：24Q1为2.243

#### 分析
公司的总资产周转率较低，表明资产运转效率有待提高。存货和应收账款的周转

''

In [20]:
question = "贵州茅台近期的舆情和财报数据怎么样"
stock_agent = StockAgent("stock_agent")
stock_agent.agent_work_flow(question)

```json
{
    "stock_name": "贵州茅台"
}
```


```json
{
    "question_type": ["财务分析", "股吧舆情"]
}
```

['财务分析', '股吧舆情']
sh600519的最近财务报表数据...
正在访问链接...
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=sh600519&color=b#/cwfx
数据长度:44
开始获取评论....
https://guba.eastmoney.com/list,sh600519_1.html
总字数:1530
根据提供的信息，我将分别从财务数据和舆情数据两个方面来分析贵州茅台近期的表现。

### 财务数据分析

以下是贵州茅台的一些关键财务指标：

| 指标名称 | 2023年3月31日 | 2022年12月31日 | 2022年9月30日 | 2022年6月30日 | 2022年3月31日 |
| --- | --- | --- | --- | --- | --- |
| 基本每股收益(元) | 19.16 | 59.49 | 42.09 | 28.64 | 16.55 |
| 每股净资产(元) | 190.84 | 171.68 | 173.39 | 159.94 | 173.76 |
| 每股公积金(元) | 1.09 | 1.09 | 1.09 | 1.09 | 1.09 |
| 每股未分配利润(元) | 156.86 | 137.70 | 143.39 | 129.94 | 144.94 |
| 每股经营现金流(元) | 7.31 | 53.01 | 39.80 | 24.19 | 4.18 |
| 营业总收入(元) | 464.8亿 | 1506亿 | 1053亿 | 709.9亿 | 393.8亿 |
| 毛利率(%) | 92.61 | 91.96 | 91.71 | 91.80 | 92.60 |

#### 分项分析：
1. **盈利能力**：贵州茅台的基本每股收益持续增长，显示出公司盈利能力较强。毛利率保持在90%以上，说明公司产品具有很高的附加值。
2. **成长能力**：营业总收入同比增长率在18%左右，归属

''